In [1]:
# Import libraries

import pandas as pd
import numpy as np
import io
import os
import sys
import sklearn
from sklearn.ensemble import RandomForestClassifier

In [2]:
# read file
CHR4_input = pd.read_csv('INPUT_FILE',  sep = " ")
# Set index
CHR4_input = CHR4_input.set_index('ACC')
# .T
CHR4_input = CHR4_input.T

In [3]:
CHR4_input = CHR4_input.astype('int8')

In [4]:
CHR4_input

ACC,Bvchr1.sca001_8490,Bvchr1.sca001_8516,Bvchr1.sca001_8518,Bvchr1.sca001_8832,Bvchr1.sca001_8931,Bvchr1.sca001_8970,Bvchr1.sca001_9236,Bvchr1.sca001_9251,Bvchr1.sca001_9318,Bvchr1.sca001_10240,...,Bvchr9_un.sca018_370300,Bvchr9_un.sca018_370474,Bvchr9_un.sca018_370475,Bvchr9_un.sca018_370527,Bvchr9_un.sca018_370647,Bvchr9_un.sca018_370714,Bvchr9_un.sca018_370721,Bvchr9_un.sca018_370749,Bvchr9_un.sca018_370762,Bvchr9_un.sca018_370778
IDBBNR352_,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
BETA_2322_,0,0,0,0,2,0,2,0,0,2,...,0,0,0,0,2,0,0,0,0,2
PI_604511_,0,0,0,0,2,0,0,0,2,2,...,0,1,0,0,0,0,0,2,0,0
Ames2649_,2,2,2,0,0,2,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
PI_116809_,1,1,1,0,0,0,0,1,0,2,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Spinat,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
UMSBv____,2,2,2,0,0,2,0,0,-1,2,...,0,0,0,0,2,0,0,0,0,2
WB42_____,0,0,0,0,2,0,0,0,1,2,...,0,0,0,0,2,0,0,0,0,2
YMoBv____,2,2,2,0,0,2,0,0,-1,2,...,0,0,0,0,0,0,0,0,0,0


In [5]:
# creates lists for groups
# ADAs
ADAs = open("ACCESSION_LIST", "r")
content = ADAs.read()
ADAs_list = content.split("\n")
del ADAs_list[-1]
ADAs.close()
# MACROs
Macro = open("ACCESSION_LIST", "r")
content = Macro.read()
Macro_list = content.split("\n")
del Macro_list[-1]
Macro.close()
# MARs
MARs = open("ACCESSION_LIST", "r")
content = MARs.read()
MARs_list = content.split("\n")
del MARs_list[-1]
MARs.close()

In [6]:
# Combine groups
MacAda = Macro_list + ADAs_list
MarAda = ADAs_list + MARs_list

In [7]:
CHR4_input = CHR4_input.loc[MarAda]

In [8]:
CHR4_input['ADA'] = CHR4_input.index.isin(ADAs_list)

In [9]:
y = CHR4_input['ADA'].values
X = CHR4_input.drop(['ADA'], axis=1).values

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y ,random_state = 42, stratify = y)

In [14]:
from sklearn.model_selection import RandomizedSearchCV# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 100, num = 1)]
# Number of features to consider at every split
max_features = [0.05]
# Maximum number of levels in tree
max_depth = [10,70]
# Minimum number of samples required to split a node
min_samples_split = [20]
# Minimum number of samples required at each leaf node
min_samples_leaf = [2]
# Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [15]:
rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 2, cv = 3, verbose = 2 ,  random_state=38, n_jobs = 30)

In [16]:
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=2,
                   n_jobs=30,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 70],
                                        'max_features': [0.05],
                                        'min_samples_leaf': [2],
                                        'min_samples_split': [20],
                                        'n_estimators': [100]},
                   random_state=38, verbose=2)

In [20]:
rf_random.best_params_

In [17]:
rf_random.best_estimator_.score(X_test,y_test)

0.9859154929577465